In [114]:
import pandas as pd
import pulp
import os


In [115]:
print(os.getcwd())
#获取当前路径

f:\------------------OWN\VSCode\Python\Math\MathClub\51club\B


In [116]:
def read_excel_data(filepath):
    demands = {}
    paths = {}
    demands_dataframe = pd.read_excel(filepath,sheet_name='Demands')
    paths_dataframe = pd.read_excel(filepath, sheet_name='Paths')

    for index, row in demands_dataframe.iterrows():
        origin_destination = (row['Start'], row['End'])
        demand_value = row['Demand']
        demands[origin_destination] = demand_value * 1.0

    for index, row in paths_dataframe.iterrows():
        key = (row['Start'], row['End'])
        if key not in paths:
            paths[key] = []
        paths[key].append(row['Path'].split(','))
    return demands,paths

In [117]:
#计算单个路径的可通过概率
def calculate_path_accessibility(path,uniform_possibility):
    path_possibility = 1.0
    path_length = len(path) - 1
    for i in range(path_length):
        path_possibility *= (1.0-uniform_possibility)
    return path_possibility

In [124]:
filepath = r'TDN2.xlsx'
demands,paths = read_excel_data(filepath)
uniform_possibility = 0.22

In [119]:
#创建问题
problem = pulp.LpProblem("Maximized_Reachability",pulp.LpMaximize)

#决策变量
traffic_flows = {}
#为每条路径设置一个变量
for origin_destination,paths_list in paths.items():
    for path in paths_list:
        var_name = f"Path_{origin_destination}_{tuple(path)}"
        key = (origin_destination,tuple(path))
        ###在空字典中创建一个以key为键值，值为等号后面的东西的 新的键值对
        traffic_flows[key] = pulp.LpVariable(var_name, lowBound=0, cat='Continuous')

#求出每条路径的概率存入字典
reachabilities = {}
for origin_destination,paths_list in paths.items():
    for path in paths_list:
        var_name = f"Path_{origin_destination}_{tuple(path)}"
        key = (origin_destination,tuple(path))
        ###在空字典中创建一个以key为键值，值为等号后面的东西的 新的键值对
        reachabilities[key] = calculate_path_accessibility(path,uniform_possibility)

In [120]:
#需要优化的表达式
for origin_destination,paths_list in paths.items():
    for path in paths_list:
        problem += reachabilities[origin_destination,tuple(path)] * traffic_flows[origin_destination,tuple(path)]

#约束条件(每个变量的范围，小于demands)
#1
for origin_destination,paths_list in paths.items():
    problem += pulp.lpSum([traffic_flows[origin_destination,tuple(path)] for path in paths_list]) == demands[origin_destination]
#2
for origin_destination,paths_list in paths.items():
    for path in paths_list:
        problem += traffic_flows[origin_destination,tuple(path)] <= demands[origin_destination]

d:\Python\PyEnvs\MathClub\lib\site-packages\pulp\pulp.py:1668: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [121]:
status = problem.solve()
print(status)

1


In [122]:
for i in problem.variables():
    print(i.name,"=",i.varValue)
objective_value = pulp.value(problem.objective)
print(objective_value)

Path_(1,_5)_('1',_'2',_'3',_'6',_'5') = 50.0
Path_(1,_5)_('1',_'2',_'3',_'6',_'9',_'8',_'5') = 0.0
Path_(1,_5)_('1',_'2',_'5') = 50.0
Path_(1,_5)_('1',_'4',_'5') = 0.0
Path_(1,_5)_('1',_'4',_'7',_'8',_'5') = 0.0
Path_(1,_6)_('1',_'2',_'3',_'6') = 50.0
Path_(1,_6)_('1',_'2',_'5',_'6') = 100.0
Path_(1,_6)_('1',_'4',_'5',_'2',_'3',_'6') = 0.0
Path_(1,_6)_('1',_'4',_'5',_'6') = 0.0
Path_(1,_6)_('1',_'4',_'7',_'8',_'5',_'6') = 0.0
Path_(1,_8)_('1',_'2',_'3',_'6',_'5',_'8') = 150.0
Path_(1,_8)_('1',_'2',_'5',_'4',_'7',_'8') = 50.0
Path_(1,_8)_('1',_'2',_'5',_'8') = 0.0
Path_(1,_8)_('1',_'4',_'5',_'8') = 0.0
Path_(1,_8)_('1',_'4',_'7',_'8') = 0.0
Path_(1,_9)_('1',_'2',_'3',_'6',_'9') = 50.0
Path_(1,_9)_('1',_'2',_'5',_'6',_'9') = 130.0
Path_(1,_9)_('1',_'4',_'5',_'6',_'9') = 0.0
Path_(1,_9)_('1',_'4',_'5',_'8',_'9') = 0.0
Path_(1,_9)_('1',_'4',_'7',_'8',_'9') = 0.0
Path_(2,_4)_('2',_'1',_'4') = 0.0
Path_(2,_4)_('2',_'3',_'6',_'5',_'4') = 0.0
Path_(2,_4)_('2',_'3',_'6',_'9',_'8',_'5',_'4') = 5